In [5]:
%load_ext autoreload
%autoreload 2

# Word (.docx) to CSV

In [2]:
from translation_helpers import docx_to_csv

In [3]:
word_file = "../../data/Parkinson/en/7-1_script_interview_clinique_1 English.docx"

docx_to_csv(word_file, data_directory="../../data")
# To save in the main results directory
#docx_to_csv(word_file, output_directory="../../results", data_directory="../../data")

'results\\Parkinson\\en\\7-1_script_interview_clinique_1 English.csv'

# Translation

In [1]:
from transformers import SeamlessM4Tv2ForTextToText, AutoProcessor
import torch
from translation_helpers import translate_by_row_csv, translate_with_chunks

c:\Users\david\anaconda3\envs\seam\lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

True
1
NVIDIA GeForce GTX 1050 Ti


In [3]:
model = SeamlessM4Tv2ForTextToText.from_pretrained("facebook/seamless-m4t-v2-large")
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
cuda = True

if cuda and torch.cuda.is_available():
    model = model.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Check the max position embeddings (equivalent to max token length)
max_length = model.config.max_position_embeddings
print(f"Max token length: {max_length}")

model.config.max_new_tokens

Max token length: 4096


256

### Chuncks translations

In [7]:
from translation_helpers import split_text_into_chunks, translation, parse_and_write_translated_text

In [8]:
source_lang = "fra" # French
target_lang = "eng" # English
#path_file = "../../results/Parkinson/fr/7-2_script_interview_clinique_1_13-08-2020.csv" # long file
path_file = "../../results/Parkinson/fr/7-1_script_interview_clinique_3_21-08-2020.csv" # short file

In [9]:
import os
def get_files(directory, extensions):
    """Get a list of files in the specified directory and its subdirectories with given extensions."""
    files = []

    for root, dirs, files_in_dir in os.walk(directory):
        for file in files_in_dir:
            if any(file.endswith(ext) for ext in extensions):
                files.append(os.path.join(root, file))
                
    return files

# def to see the max numbers of tokens per row in the csv file
files = get_files("../../results/Parkinson/fr", [".csv"])

for file in files:
    max_tokens = 0
    with open(file, "r") as f:
        index = 0
        for line in f:
            index += 1
            tokens = len(line.split())
            if tokens > max_tokens:
                max_tokens = tokens
                index_max = index
    print(file," max tokens: ",max_tokens,"at line: ",index_max)

../../results/Parkinson/fr\7-1_script_interview_clinique_1.csv  max tokens:  104 at line:  226
../../results/Parkinson/fr\7-1_script_interview_clinique_2_21-08-2020.csv  max tokens:  109 at line:  55
../../results/Parkinson/fr\7-1_script_interview_clinique_3_21-08-2020.csv  max tokens:  72 at line:  32
../../results/Parkinson/fr\7-1_script_interview_clinique_3_21-08-2020_fra_to_eng.csv  max tokens:  1 at line:  1
../../results/Parkinson/fr\7-1_script_interview_clinique_4_21-08-2020.csv  max tokens:  63 at line:  3
../../results/Parkinson/fr\7-1_script_interview_clinique_4_21-08-2020_eng.csv  max tokens:  60 at line:  3
../../results/Parkinson/fr\7-1_script_interview_clinique_5_21-08-2020.csv  max tokens:  78 at line:  55
../../results/Parkinson/fr\7-2_script_interview_clinique_1_13-08-2020.csv  max tokens:  118 at line:  51
../../results/Parkinson/fr\7-2_script_interview_clinique_2_13-08-2020.csv  max tokens:  61 at line:  2
../../results/Parkinson/fr\7-2_script_interview_clinique_3_13

In [11]:
max_length = 200 
chunks = split_text_into_chunks(path_file, max_length, processor, source_lang)
print(f"Number of chunks: {len(chunks)}")
for text in chunks:
    tokens = processor(text, return_tensors="pt", src_lang=source_lang)
    print(f"Number of tokens: {len(tokens['input_ids'][0])}")

# Translate the chunks
translated_chunks = []
for chunk in chunks:
    translated_text = translation(source_lang, target_lang, chunk, model, processor, True)
    translated_chunks.append(translated_text)

Number of chunks: 10
Number of tokens: 190
Number of tokens: 170
Number of tokens: 176
Number of tokens: 184
Number of tokens: 172
Number of tokens: 118
Number of tokens: 172
Number of tokens: 178
Number of tokens: 196
Number of tokens: 76


In [13]:
chunks

["<<SPEAKER:Examinatrice>>\nDonc, toujours concernant cette sensation de présence dans le dos, ce vieux monsieur en costume gris. Est-ce que vous avez l'impression, parfois, que c'est cette personne, cette présence essaye d'interagir avec vous, de communiquer avec vous ?\n\n<<SPEAKER:Patiente>>\nPas vraiment, parce qu’ils parlent quand même entre eux, mais très doucement.\n\n<<SPEAKER:Examinatrice>>\nLe monsieur en gris ?\n\n<<SPEAKER:Patiente>>\nOui mais il parle avec les autres qui sont avec ce monsieur gris. Je le vois, je le vois chaque soir, mais en principe il est tout seul.\n\n<<SPEAKER:Examinatrice>>\nIl est tout seul. D'accord. Donc, il n’essaye pas de vous parler ?",
 "<<SPEAKER:Patiente>>\nNon.\n\n<<SPEAKER:Examinatrice>>\nEst-ce que vous avez l'impression qu'il essaie de lire dans vos pensées ou qu'il sait déjà à quoi vous pensez ?\n\n<<SPEAKER:Patiente>>\nEuh, je ne sais pas. Des fois, je me demande : Est-ce que qu’ils savent déjà ce qui se passe demain ?\n\n<<SP

In [12]:
translated_chunks

["So, always about this feeling of presence in the back, this old man in a gray suit. Do you sometimes get the impression that it's this person, this presence trying to interact with you, to communicate with you? So, not really, because they're still talking to each other, but very softly. So, the gray man, yes, but he's talking to the others who are with this gray man. I see him, I see him every night, but basically he's all alone.",
 "No. Do you have the impression that he's trying to read your mind or that he already knows what you're thinking? Well, I don't know. Sometimes I wonder: Do they already know what's going to happen tomorrow?",
 '"SPEAKER: Patient". "Yes, something like that? Predicting?" "Yes, or maybe he\'s doing a calculation. A calculation to find out an average?" "I don\'t know what to say". "Well, I don\'t know. Maybe it\'s nothing". "SPEAKER: Examiner". "No, but you say what you think". "So how do you explain this phenomenon?" "With the old man?" "SPEAKER: Examiner

In [ ]:

# Parse and write the translated text
output_csv = f"{os.path.splitext(path_file)[0]}_{source_lang}_to_{target_lang}.csv"
parse_and_write_translated_text(translated_chunks, output_csv)

### Translation row by row CSV

In [21]:
source_lang = "fra" # French
target_lang = "eng" # English
path_file = "../../results/Parkinson/fr/7-1_script_interview_clinique_3_21-08-2020.csv"

translate_by_row_csv(path_file, source_lang, target_lang, model, processor, cuda)

Translating: 100%|██████████| 19/19 [01:56<00:00,  6.12s/row]
